# Setting up the environment for Molecular Dynamics Simulations
## First, we need to install all necessary libraries and packages for our simulation. The main packages we will be installing are:

Anaconda (https://docs.conda.io/en/latest/miniconda.html)
OpenMM (https://openmm.org/)
MDAnalysis (https://www.mdanalysis.org/)
Numpy (https://numpy.org/)
Pandas (https://pandas.pydata.org/)
Matplotlib (https://matplotlib.org/)

### Only Execute the Next Two Cells if Using Google Colab

In [ ]:
#@title ### **Import Google Drive**
#@markdown Click in the "Run" buttom to make your Google Drive accessible. Only run this command if in Google Colab!
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title **Check if you correctly allocated GPU nodes**
#@markdown Again, only run this command if you are running this code in Google Colab!

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [1]:
#@title **Install dependencies**
#@markdown It will take a few minutes, please, drink a coffee and wait.
# install dependencies
#%%capture
!pip -q install py3Dmol
!pip install git+https://github.com/pablo-arantes/biopandas
!conda install mamba -c conda-forge -y
!mamba install -c conda-forge gcc=12.1.0 -y
!mamba install openmm -c conda-forge -y
!pip install --upgrade MDAnalysis
!pip install seaborn

  Cloning https://github.com/pablo-arantes/biopandas to /tmp/pip-req-build-q8pwcaow
  Running command git clone --filter=blob:none --quiet https://github.com/pablo-arantes/biopandas /tmp/pip-req-build-q8pwcaow
  Resolved https://github.com/pablo-arantes/biopandas to commit 107e1a12491478242d373732a6fb3416b2569266
  Preparing metadata (setup.py) ... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0



# All requested packages already installed.


Looking for: ['gcc=12.1.0']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
pkgs/main/linux-64 ━━━━━━━━━━━╸━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
pkgs/main/noarch  

In [1]:
#@title **Load dependencies**
import sys
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import urllib.request  
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis.dielectric import DielectricConstant
import py3Dmol
import platform
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
from matplotlib import colors
from IPython.display import set_matplotlib_formats

#conversion factors to get to correct units
mass_to_kg = 1.66054e-27
angstrom_to_m = 1e-10
ps_to_s = 1e-12
avogadro = 6.022e23

#file locations
tip3p_traj = "tip3p-traj.dcd"
tip3p_log = "tip3p_log.csv"
tip3p_pdb = "tip3p_minimized.pdb"

tip4p_traj = "tip4p-traj.dcd"
tip4p_log = "tip4p-log.csv"
tip4p_pdb = "tip4p_minimized.pdb"

# Preparing & Running a Molecular Dynamics Simulation of Water

In [2]:
#@title **Cosntruct the System**
#@markdown Create a box with randomly placed TIP3P water molecules with periodic boundary conditions

forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")
modeller = Modeller(Topology(), [])
modeller.addSolvent(forcefield, boxSize=Vec3(3, 3, 3)*nanometers, model='tip3p', neutralize=True, negativeIon='Cl-', positiveIon='Na+')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
print("Number of TIP3P water molecules in box in (3 nm)^3 cubic box: ", len(modeller.getPositions()))

Number of TIP3P water molecules in box in (3 nm)^3 cubic box:  2661


In [3]:
#@title **Add termostat & barostat. Run energy minimization.**
#@markdown Here we define the temperature of the system (300 K), the pressure (1 bar), assign random velocities to particles/molecules, and define our time step (dt) to be 2 fs. Then, we run a gradient descent energy minimization on the system.

system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(tolerance=0.1*kilojoule/mole, maxIterations=100000)

positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(tip3p_pdb, 'w'))


In [9]:
#@title **Visualize the System**

view = py3Dmol.view(width=500, height=500)
view.addModel(open( 'minimized.pdb' ,'r').read(),'pdb')
view.setStyle({'stick':{}} )
view.addSurface(py3Dmol.SAS, {'opacity': 0.3, 'color':'blue'})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
#@title Run a Production Molecular Dynamics Simulation
#@markdown This simulation should produce a ~45 MB trajectory file called traj.dcd in your Google Drive. The trajectory will contain dynamics information of water molecules over the course of 30 ns.

simulation.reporters = []
#record positions every 100 steps
simulation.reporters.append(DCDReporter(tip3p_traj, 100))
simulation.reporters.append(StateDataReporter(tip3p_log, 500, step=True, time=True,
                                              temperature=True, kineticEnergy=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter(sys.stdout, 1000, step=True, time=True, elapsedTime=True))

#run the simulation for defined number of steps with timestep of 2 fs
simulation.step(15000)

#"Step","Time (ps)","Elapsed Time (s)"
1000,2.0000000000000013,0.0001227855682373047
2000,3.999999999999781,14.534170150756836
3000,5.999999999999561,29.148587465286255
4000,7.999999999999341,43.856016874313354
5000,10.000000000000009,57.13489103317261
6000,12.000000000000677,71.42403841018677
7000,14.000000000001345,84.63480949401855
8000,16.00000000000201,97.81077456474304
9000,18.000000000000902,111.2475335597992
10000,19.999999999999794,124.69748187065125
11000,21.999999999998685,137.98349356651306
12000,23.999999999997577,151.367360830307
13000,25.99999999999647,164.75861763954163
14000,27.99999999999536,178.2820873260498
15000,29.99999999999425,192.87894535064697


# Analyzing the Simulation

In [5]:
#@title Plotting System Temperature
#@markdown 


tip3p_data = pd.read_csv(tip3p_log)


In [6]:
#@title System Temperature and the Equipartition Theorem
#@markdown 

print("Average Temperature: ", tip3p_data["Temperature (K)"].mean())
print("Average Kinetic Energy: ", tip3p_data["Kinetic Energy (kJ/mole)"].mean())
print("(<KE>/<T>)*(1/Avogadro's Number): ", (tip3p_data["Kinetic Energy (kJ/mole)"].mean()/tip3p_data["Temperature (K)"].mean())/avogadro)

Average Temperature:  295.02916334704986
Average Kinetic Energy:  6523.777302296474
(<KE>/<T>)*(1/Avogadro's Number):  3.6719218420754796e-23


In [7]:
#@title Calculate the density and dielectric constant of water
#@markdown 

universe = mda.Universe(tip3p_pdb, tip3p_traj)
water = universe.select_atoms("resname HOH")

#mass density of water - first calculate total mass of water in box
water_mass = 0
for i in range(len(water)):
    water_mass += water[i].mass * mass_to_kg

av_density = 0
for p in range(0, len(universe.trajectory)):
    universe.trajectory[p]

    volume = (universe.trajectory[p].dimensions[0]*angstrom_to_m) * (universe.trajectory[p].dimensions[1]*angstrom_to_m) * (universe.trajectory[p].dimensions[2]*angstrom_to_m)
    av_density += water_mass/volume

av_density = av_density/len(universe.trajectory)
print("The average density of TIP3P water is: ", av_density, " [kg/m^3]")


"""
#dielectric constant of water
diuniverse = mda.Universe('tip3p-state.xml', 'traj.dcd')
diel = DielectricConstant(diuniverse.atoms)
diel.run()
print(diel.results.eps_mean)
"""

/home/brian/miniconda3/envs/jupyter-notes/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


The average density of TIP3P water is:  980.7650476214585  [kg/m^3]


"\n#dielectric constant of water\ndiuniverse = mda.Universe('tip3p-state.xml', 'traj.dcd')\ndiel = DielectricConstant(diuniverse.atoms)\ndiel.run()\nprint(diel.results.eps_mean)\n"

# A Different Representation of Water: TIP4P/2005

In [76]:
#@title Simulate TIP4P/2005 water model and calculate the density and dielectric constant to compare with TIP3P

#system setup
forcefield = ForceField("charmm36.xml", "charmm36/tip4p2005.xml")
modeller = Modeller(Topology(), [])
modeller.addSolvent(forcefield, boxSize=Vec3(3, 3, 3)*nanometers, model='tip4pew', neutralize=True, negativeIon='Cl-', positiveIon='Na+')
modeller.addExtraParticles(forcefield)
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
print("Number of TIP4P/2005 water molecules in box in (3 nm)^3 cubic box: ", len(modeller.getPositions()))

#simulation details and energy minimization
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(tolerance=0.1*kilojoule/mole, maxIterations=100000)

positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(tip4p_pdb, 'w'))

#running simulation
simulation.reporters = []
simulation.reporters.append(DCDReporter(tip4p_traj, 100))
simulation.reporters.append(StateDataReporter(tip4p_log, 500, step=True, time=True,
                                              temperature=True, kineticEnergy=True, elapsedTime=True))

simulation.step(15000)

universe = mda.Universe(tip4p_pdb, tip4p_traj)
water = universe.select_atoms("resname HOH")

#mass density of water - first calculate total mass of water in box
water_mass = 0
for i in range(len(water)):
    water_mass += water[i].mass * mass_to_kg

av_density = 0
for p in range(0, len(universe.trajectory)):
    universe.trajectory[p]

    volume = (universe.trajectory[p].dimensions[0]*angstrom_to_m) * (universe.trajectory[p].dimensions[1]*angstrom_to_m) * (universe.trajectory[p].dimensions[2]*angstrom_to_m)
    av_density += water_mass/volume

av_density = av_density/len(universe.trajectory)
print("The average density of TIP3P water is: ", av_density, " [kg/m^3]")


"""
#dielectric constant of water
diuniverse = mda.Universe('tip3p-state.xml', 'traj.dcd')
diel = DielectricConstant(diuniverse.atoms)
diel.run()
print(diel.results.eps_mean)
"""

Number of TIP4P/2005 water molecules in box in (3 nm)^3 cubic box:  3544
